# STA 141B Data & Web Technologies for Data Analysis


### Lecture 9, 2/6/25, APIs


### Today's topics

- Undocumented APIs

### Ressources
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

### Recap: HTTP

A response to an HTTP request always includes a status code that summarizes whether the request was successful. Wikipedia has a full [list of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). Generally,

* 200-299: Your request succeeded.
* 300-399: You need to take further action to complete the request.
* 400-499: Your request wasn't valid (you made a mistake). You've probably seen 404 before!
* 500-599: Your request failed (the server made a mistake).

In [ ]:
import requests

### Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

 - [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

You can identify these websites by looking at requests in your browser's developer tools. For Firefox and Chrome these can be accessed (Windows: <kbd>Ctrl</kbd> + <kbd>i</kbd>; MacOS: <kbd>&#8984;</kbd> + <kbd>&#8997;</kbd> + <kbd>i</kbd>).

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

Let's reverse engineer the Yolo County Health Inspections web API so that we can get data about local restaurants.

In [ ]:
import numpy as np
import pandas as pd
import requests

In [ ]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [ ]:
result = requests.post(url, 
                       data = {
    'FacilityName': "Ali Baba"
})

In [ ]:
result.text

Check the [docs](https://requests.readthedocs.io/en/latest/api/?highlight=post#requests.post) for `requests`!

In [ ]:
result.url

In [ ]:
result.json()

Lets investigate this further. The second request uses the `FacilityID` as parameter. 

In [ ]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/programs'
result = requests.get(url, params = {
    'FacilityId': 'FA0001973', 
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4'
})
result.raise_for_status()
result.json()

In [ ]:
result.url

We are interested in the inspections text, for which we have to provide the `ProgramID` parameter. 

In [ ]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections'

In [ ]:
result = requests.get(url, params = {
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
    'ProgramId': 'PR0000674'
})
result.raise_for_status()

In [ ]:
results = result.json()
results

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
results_df['violations'][1]

In [ ]:
results_df['violations'][1][0]['v_memo']

In [ ]:
len(results_df['violations'][1])

In [ ]:
violations = [
    results_df['violations'][1][i]['violation_description'] for i in range(len(results_df['violations'][1]))
]
violations

In [ ]:
{'Ali Baba': violations}

How can we generalize this procedure? 

In [ ]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [ ]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"
}, 
                     data = {
    "FacilityName": "Ali Baba", 
})
result.raise_for_status()

In [ ]:
result.json()

In [ ]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
              data = {
    "FacilityName": "a", 
})
result.json()

In [ ]:
pd.DataFrame(result.json())

Lets write a pipeline. 

In [ ]:
def fetch_violations(ProgramId):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': ProgramId
    })
    result.raise_for_status()
    results = result.json()
    results_df = pd.DataFrame(results)
    violations = [
        results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))
    ]
    return(violations)

In [ ]:
fetch_violations('PR0000623') # for in-n-out

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0000623'
    })

In [ ]:
results = result.json()
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0024103'
    })

In [ ]:
result.text

In [ ]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    })
    result.raise_for_status()
    ProgramId = result.json()[0]['ProgramId']
    return(ProgramId)

In [ ]:
fetch_ProgramId('FA0001345')

In [ ]:
def fetch_FacilityID(letter):
    result = requests.post('https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities?', 
                           params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
                           data = {
    "FacilityName": letter, 
    })
    facility_table = pd.DataFrame(result.json())[['FacilityId', 'FacilityName']]
    return(facility_table)

In [ ]:
fetch_FacilityID('A&B LIQUOR')

In [ ]:
import time

In [ ]:
[letter for letter in map(chr, range(97, 99))]

In [ ]:
x = {}
type(x)

In [ ]:
def get_violations(): 
    violations = {}
    for letter in map(chr, range(97, 99)): # map(chr, range(97, 123)) takes too long
        time.sleep(0.05) # sleep until making a request for each letter
        facility_table = fetch_FacilityID(letter)
        for index in range(facility_table.shape[0]): # for all facilities returned for this letter
            FacilityId, FacilityName = facility_table.iloc[index]
            time.sleep(0.1) # sleep again for each individual request
            ProgramId = fetch_ProgramId(FacilityId)
            print(FacilityName)
            violations[FacilityName] = fetch_violations(ProgramId)
    return(violations)

In [ ]:
violations = get_violations()

In [ ]:
x = {'key': 'value'}
x['keyy']

In [ ]:
fetch_FacilityID('A&B LIQUOR')

In [ ]:
fetch_ProgramId('FA0001345')

In [ ]:
ProgramId = fetch_ProgramId('FA0001345')            
ProgramId

In [ ]:
fetch_violations('PR0000623')

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0000623'
})
result.raise_for_status()

In [ ]:
results = result.json()
results

Lets check this in the browser! 

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': 'FA0001345'
    }).json()
[result[i]['ProgramId'] for i in range(len(result))]

In [ ]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    }).json()
    ProgramId = [result[i]['ProgramId'] for i in range(len(result))]
    return(ProgramId)

In [ ]:
fetch_ProgramId('FA0001345')

In [ ]:
def fetch_violations(ProgramId_list):
    violations = []
    for ProgramId in ProgramId_list: 
        result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
            'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
            'ProgramId': ProgramId
        }).json()
        results_df = pd.DataFrame(result)
        if not results_df.empty: # only append violations if there are any
            violations.extend(
                [results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))]
            )
    return(violations)

In [ ]:
fetch_violations(['PR0000623', 'PR0069422'])

In [ ]:
violations = get_violations()

In [ ]:
violations

#### Safeway

Check the [docs](https://requests.readthedocs.io/en/latest/api/?requests.get)!

In [ ]:
url = 'https://www.safeway.com/abs/pub/xapi/pgmsearch/v1/search/products'
params = {
    'request-id': 5561736793073137191,
    'q': 'eggs',
    'rows': 30,
    'start': 0,
    'search-type': 'keyword',
    'storeid': 3132,
    'featured': 'true',
    'url': 'https://www.safeway.com',
    'pageurl': 'https://www.safeway.com', 
    'search-uid': 'uid%3D3640904575678%3Av%3D12.0%3Ats%3D1674581210532%3Ahc%3D3', 
    'pagename': 'search',
    'dvid': 'web-4.1search',
}
header = {
    #'accept': 'application/json, text/plain, */*',
    #'accept-encoding': 'gzip, deflate, br, zstd',
    #'accept-language': 'en-US,en;q=0.9',
    #'cache-control': 'no-cache',
    'Cookie': "visid_incap_1610353=NpQjFt3XShCg5HmsJYLwrxWpGmcAAAAAQUIPAAAAAABTbH3K+b6Xlzm4bbIilz0A; OptanonAlertBoxClosed=2024-10-24T20:13:12.155Z; akacd_PR-bg-www-prod-safeway=3914245866~rv=80~id=f9720057ac5012141d9501502823ae32; nlbi_1610353=M32aK2/TRgycgjWj6eNT2gAAAABGpxl4Fz9uGtERK5SQr0G+; incap_ses_1357_1610353=x9+QQFPW5kOEJ75beQjVEupbhWcAAAAANJR7ORNIqIiNK+nGTcaNmw==; AMCVS_A7BF3BC75245ADF20A490D4D%40AdobeOrg=1; ACI_S_ECommBanner=safeway; abs_gsession=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22Selection%22%3A%22default%22%2C%22preference%22%3A%22J4U%22%2C%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%2C%22SHOP%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%7D%7D; ACI_S_abs_previouslogin=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22Selection%22%3A%22default%22%2C%22preference%22%3A%22J4U%22%2C%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%2C%22SHOP%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%7D%7D; SWY_SYND_USER_INFO=%7B%22storeAddress%22%3A%22%22%2C%22storeZip%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%2C%22preference%22%3A%22J4U%22%7D; ACI_S_ECommSignInCount=0; at_check=true; SAFEWAY_MODAL_LINK=; SWY_SHARED_SESSION_INFO=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22preference%22%3A%22J4U%22%2C%22Selection%22%3A%22default%22%2C%22wfcStoreId%22%3A%225799%22%2C%22userData%22%3A%7B%7D%2C%22grsSessionId%22%3A%2265868f2b-f89f-4a7a-9edc-f8607c5728a9%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22storeId%22%3A%223132%22%2C%22zipcode%22%3A%2294611%22%2C%22userData%22%3A%7B%7D%7D%2C%22SHOP%22%3A%7B%22storeId%22%3A%223132%22%2C%22zipcode%22%3A%2294611%22%2C%22userData%22%3A%7B%7D%7D%7D%7D; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Jan+13+2025+10%3A31%3A10+GMT-0800+(Pacific+Standard+Time)&version=202409.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=a294f937-8674-4c26-9178-9f1b60f6b59b&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0004%3A1%2CC0003%3A1&AwaitingReconsent=false&intType=3&geolocation=US%3BCA; nlbi_1610353_2147483392=5gvNQA0KaE9HVNxe6eNT2gAAAABCup9oWsSAk1ZDDN2TmX8R; reese84=3:ExNoTZgnHBodyq0jNmWs8g==:J4VcV6yDVCf3FV56rwTIU0ZXkTM8sNmwT9oLEQRGS7tecwRkhajCAwJkTzB8jP83kx7XvlxPaSjkfhEOeHL45QvGi6QlLt1eIzP5/FW9ya+sDicCoCaxSCUOJEgQNKLdzYYAhbpZ8c2qMWT4CYUIYE94uC6MQqu7NIoDGszKTcBq6GYnz6j6GgmNCffd6J43IBdcMTMMh+9WMDk3+FAhIQ3p1HpHGPGuy39PSxe1VLglWGMrcmdax81dwmzRsGKkA2BKcq5Zh8bdEZWwHIT02lXZxgglCcRMdZN77huBciKdRmC6Ie9M6CHLYogzRoVgH0QEuaCBDzx+3TzrmimnffHp4fwe2wcraMfyyvzEViz8PPH9hrEJaRiLRb10QfcxsIASJ1oe06T2tl+GFd6LlHO72GKSgm1FyJLpcIQAj7ot8a9k6SJVTd65bLawwhiPUOR9V3Reh1zVU40hUYVYYgK0WQAHT2onMOhwvk4Pic5vdxSO7iKW0HxX+jqZIq79WRw1c/qcVrAFg7veHbQsJw==:qij+BfrUCJWyBDLfyP92iy/QSXKrDa2fVrAr/ygSiSY=; AMCV_A7BF3BC75245ADF20A490D4D%40AdobeOrg=179643557%7CMCIDTS%7C20102%7CMCMID%7C88307423134550140093054020464612948435%7CMCOPTOUT-1736800272s%7CNONE%7CvVersion%7C5.5.0; mbox=PC#ce30384f188648868a639d719bb80350.35_0#1800037874|session#c608f2b05a384af1a61447777b4fc45d#1736794934",
    'Ocp-Apim-Subscription-Key': '5e790236c84e46338f4290aa1050cdd4'
    #'pragma': 'no-cache',
    #'priority': 'u=1, i',
    #'referer': 'https://www.safeway.com/shop/search-results.html?q=eggs',
    #'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    #'sec-ch-ua-mobile':'?0',
    #'sec-ch-ua-platform':"macOS",
    #'sec-fetch-dest': 'empty',
    #'sec-fetch-mode': 'cors',
    #'sec-fetch-site': 'same-origin',
    #'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

# try without cookie and subscription key! 

In [ ]:
results = requests.get(url, params = params, headers = header)
results.json()

In [ ]:
s = requests.Session()
s.cookies

In [ ]:
s.get('https://www.safeway.com/')
s.cookies

In [ ]:
r=s.get(url, params=params, headers = {'Ocp-Apim-Subscription-Key': '5e790236c84e46338f4290aa1050cdd4'}) #error! 

In [ ]:
url2 = 'https://www.safeway.com/abs/pub/xapi/search/products'

r = requests.get(url2, params, headers = {'Ocp-Apim-Subscription-Key': 'e914eec9448c4d5eb672debf5011cf8f'}) #error! 
r.json()

### Summary 

- Check the query type, header and params using the developer tools 
- Often, multiple API queries are made to display one result 